### Install Required Packages


# Install Groq
pip install groq  

# Install Gradio and Groq


In [ ]:
!pip install groq
!pip install gradio groq



### Groq API Example

This script shows how to:
- Enter and store your **Groq API key** securely.  
- Initialize the Groq client.  
- Send a test message (`"Hello from Groq!"`) to the `llama3-8b-8192` model.  
- Stream and print the response.
⚠️ Each user must create their **own Groq API key** to run this code.

In [ ]:
import os
from groq import Groq

# Ask you for the key once (input stays local to you, not shared)
api_key = input("Enter your GROQ API key: ").strip()

# Store it in environment variable
os.environ["GROQ_API_KEY"] = api_key

# Initialize client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# Example completion
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[{"role": "user", "content": "Hello from Groq!"}],
    stream=True,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


### Groq Chatbot Example

This script creates a simple **chatbot** using the Groq API.  

**What it does:**
- Prompts the user for their **Groq API key** (each user needs their own key).  
- Stores the key securely in an environment variable.  
- Maintains a **conversation history** between user and assistant.  
- Sends user messages to the `llama3-8b-8192` model.  
- Streams and displays the assistant’s reply in real time.  
- Keeps chatting until the user types `exit` or `quit`.  


In [ ]:
import os
from groq import Groq

# Ask once for the API key
api_key = input("Enter your GROQ API key: ").strip()
os.environ["GROQ_API_KEY"] = api_key

# Initialize client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# Conversation history
messages = []

print("\n💬 Groq Chatbot (type 'exit' to quit)\n")

while True:
    # Get user input
    user_input = input("You: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break

    # Add user message
    messages.append({"role": "user", "content": user_input})

    # Get response from Groq
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=messages,
        stream=True,
    )

    print("Groq:", end=" ", flush=True)
    reply = ""
    for chunk in completion:
        content = chunk.choices[0].delta.content or ""
        print(content, end="", flush=True)
        reply += content
    print("\n")

    # Save reply into history
    messages.append({"role": "assistant", "content": reply})


### Groq Medical Chatbot Example

This script builds a simple **medical assistant chatbot** using the Groq API.  

**What it does:**
- Prompts the user for their **Groq API key** (each user must create their own).  
- Stores the key securely in an environment variable.  
- Uses a **system prompt** to guide the model as a medical assistant.  
- Provides explanations of symptoms, conditions, medicines, and health tips.  
- Always reminds users it is **not a doctor** and to consult professionals for serious issues.  
- Streams and displays answers in real time while maintaining conversation history.  
- Runs interactively until the user types `exit` or `quit`.  


In [ ]:
import os
from groq import Groq

# Ask once for the API key
api_key = input("Enter your GROQ API key: ").strip()
os.environ["GROQ_API_KEY"] = api_key

# Initialize client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# Conversation history with system prompt for medical chatbot
messages = [
    {
        "role": "system",
        "content": (
            "You are a helpful medical assistant. "
            "Provide clear, accurate, and easy-to-understand medical information. "
            "You can explain symptoms, conditions, medicines, and healthy lifestyle tips, "
            "but always remind users that you are not a doctor and they should consult a healthcare professional for serious issues."
        )
    }
]

print("\n💊 Medical Chatbot (type 'exit' to quit)\n")

while True:
    # Get user input
    user_input = input("You: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Take care! Stay healthy.")
        break

    # Add user message
    messages.append({"role": "user", "content": user_input})

    # Get response from Groq
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=messages,
        stream=True,
    )

    print("Bot:", end=" ", flush=True)
    reply = ""
    for chunk in completion:
        content = chunk.choices[0].delta.content or ""
        print(content, end="", flush=True)
        reply += content
    print("\n")

    # Save reply into history
    messages.append({"role": "assistant", "content": reply})


### Gradio Medical Chatbot (Groq + LLaMA-3)

This script builds a simple **medical chatbot** with **Groq API** and **Gradio**.  

- Requires your own **Groq API key**.  
- Uses a medical system prompt to give clear info on symptoms, conditions, medicines, and health tips.  
- Reminds users it is **not a doctor** and to consult professionals for serious issues.  
- Runs in a **web-based chat interface** with conversation history.  

⚠️ **Note:** For educational use only, not a substitute for professional medical advice.  


In [5]:
import os
import gradio as gr
from groq import Groq

# Load API key (either from input or environment)
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = input("Enter your GROQ API key: ").strip()

# Initialize client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# System prompt for medical chatbot
system_prompt = {
    "role": "system",
    "content": (
        "You are a helpful medical assistant. "
        "Provide clear, accurate, and easy-to-understand medical information. "
        "You can explain symptoms, conditions, medicines, and healthy lifestyle tips, "
        "but always remind users that you are not a doctor and they should consult a healthcare professional for serious issues."
    ),
}

# Conversation memory
messages = [system_prompt]

# Function for chatbot
def chatbot(user_input, history):
    global messages
    messages.append({"role": "user", "content": user_input})

    # Get response from Groq
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=messages,
        stream=False,  # No streaming in Gradio, just full reply
    )

    reply = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})

    # Append to history for display in Gradio
    history.append((user_input, reply))
    return history, history

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 💊 Medical Chatbot (Powered by Groq LLaMA-3)")
    chatbot_ui = gr.Chatbot()
    user_input = gr.Textbox(placeholder="Ask me about symptoms, conditions, medicines...")
    state = gr.State([])

    def user_submit(message, chat_history):
        return "", chat_history

    user_input.submit(chatbot, [user_input, state], [chatbot_ui, state]).then(
        user_submit, [user_input, chatbot_ui], [user_input, chatbot_ui]
    )

# Launch app
demo.launch()


/tmp/ipython-input-2994388557.py:48: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fb486413b379c71c89.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
